In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-17"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16176,2024-03-17,Espanha Acb,08:30,Bilbao,Joventut,2.03,1.79,161.5,1.81,1.97,1.5,1.89,1.65,rTTKeAdL,0.492611,0.558659,0.552486,0.507614,0.051270,0.4,0.2,NaN,NaN,338.866,308.088921,0.909176,186.126,75.561225,0.405968,133.808,212.526,210.18,296.46,0.0,0.0,0.0,0.0,0.088851,0.059861,0.095879,0.16,0.032,32.187500,0.000000,0.0,0.000000,2.75,0.550,1.436364,0.000000,0.0,0.000000
16177,2024-03-17,Espanha Acb,14:30,Unicaja,Real Madrid,2.29,1.63,167.5,1.84,1.94,2.5,1.92,1.38,Y3RsLwlP,0.436681,0.613497,0.543478,0.515464,0.050178,0.2,0.6,NaN,NaN,126.526,28.709070,0.226903,95.868,10.795662,0.112610,121.464,119.308,103.96,90.40,0.0,0.0,0.0,0.0,0.238107,0.037413,0.231417,1.81,0.362,3.563536,0.000000,0.0,0.000000,-0.73,-0.146,-4.315068,0.000000,0.0,0.000000
16178,2024-03-17,Eua Nba,14:00,Milwaukee Bucks,Phoenix Suns,1.74,2.17,229.5,1.93,1.93,-4.5,2.02,3.02,nRomkIb5,0.574713,0.460829,0.518135,0.518135,0.035542,0.2,0.4,NaN,NaN,201.132,23.817639,0.118418,220.848,33.606221,0.152169,170.646,209.880,185.32,242.19,0.0,0.0,0.0,0.0,0.155527,0.000000,0.280598,-1.32,-0.264,-2.803030,0.677517,0.6,-0.077517,-1.82,-0.364,-3.214286,0.585335,0.4,-0.185335
16179,2024-03-17,Eua Nba,16:30,Dallas Mavericks,Denver Nuggets,2.37,1.63,218.5,1.90,1.96,2.5,1.96,1.39,YspemdTH,0.421941,0.613497,0.526316,0.510204,0.035438,0.2,0.2,NaN,NaN,212.656,41.159389,0.193549,182.930,38.026334,0.207874,180.904,192.736,158.46,162.00,0.0,0.0,0.0,0.0,0.261630,0.021983,0.240627,2.26,0.452,3.030973,0.607562,0.7,0.092438,-0.06,-0.012,-52.500000,0.572018,0.6,0.027982
16180,2024-03-17,Eua Nba,20:00,San Antonio Spurs,Brooklyn Nets,1.88,1.98,219.5,1.86,1.92,-2.5,2.03,2.14,fR10FECA,0.531915,0.505051,0.537634,0.520833,0.036965,0.0,0.4,NaN,NaN,517.686,173.119768,0.334411,210.016,23.733205,0.113007,305.090,301.600,247.45,192.64,0.0,0.0,0.0,0.0,0.036638,0.022448,0.037305,-5.00,-1.000,-0.880000,0.348738,0.2,-0.148738,-0.71,-0.142,-6.901408,0.411785,0.3,-0.111785
16181,2024-03-17,Itália Liga A,13:00,Venezia,Milano,2.38,1.59,153.5,1.86,1.90,2.5,2.04,1.36,vmo41lHb,0.420168,0.628931,0.537634,0.526316,0.049099,0.6,0.4,NaN,NaN,121.226,19.593769,0.161630,108.616,37.506099,0.345309,122.078,120.626,120.96,171.75,0.0,0.0,0.0,0.0,0.281418,0.015045,0.282843,0.61,0.122,11.311475,0.000000,0.0,0.000000,-1.01,-0.202,-2.920792,0.724242,0.6,-0.124242
16182,2024-03-17,Itália Liga A,13:30,Scafati,Tortona,2.17,1.70,159.5,1.83,1.93,1.5,1.99,1.55,SCJihpvo,0.460829,0.588235,0.546448,0.518135,0.049065,0.8,0.4,NaN,NaN,212.284,166.849457,0.785973,137.116,19.114854,0.139406,148.800,188.628,158.40,126.38,0.0,0.0,0.0,0.0,0.171752,0.037612,0.175778,1.59,0.318,3.679245,0.000000,0.0,0.000000,-3.28,-0.656,-1.067073,0.000000,0.0,0.000000
16183,2024-03-17,Itália Liga A,14:30,Brindisi,Trento,1.77,2.05,159.5,1.86,1.92,-2.5,2.00,2.36,rurLc6GN,0.564972,0.487805,0.537634,0.520833,0.052777,0.6,0.4,NaN,NaN,261.894,208.762070,0.797124,162.558,50.151406,0.308514,187.854,162.766,132.72,244.12,0.0,0.0,0.0,0.0,0.103660,0.022448,0.116770,0.06,0.012,64.166667,0.000000,0.0,0.000000,-3.33,-0.666,-1.576577,0.000000,0.0,0.000000
16184,2024-03-17,China Cba,08:35,Jiangsu Dragons,Ningbo Rockets,1.65,2.19,194.5,1.83,1.91,-3.5,1.92,2.60,x4KXwySH,0.606061,0.456621,0.546448,0.523560,0.062682,0.4,0.6,NaN,NaN,566.312,195.531308,0.345271,553.994,238.249145,0.430057,427.490,744.538,722.50,954.60,1.0,1.0,0.0,0.0,0.198874,0.030251,0.212758,-5.00,-1.000,-0.650000,0.335882,0.1,-0.235882,-5.00,-1

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,14:00,Eua Nba,Milwaukee Bucks,Phoenix Suns,229.5,1.93,1.0,Over
1,20:00,Eua Nba,San Antonio Spurs,Brooklyn Nets,219.5,1.86,1.0,Over
2,14:30,Itália Liga A,Brindisi,Trento,159.5,1.86,1.0,Over
3,08:35,China Cba,Jiangsu Dragons,Ningbo Rockets,194.5,1.83,1.0,Over
4,08:00,Rússia Copa Da Rússia,Uralmash Ekaterinburg,MBA Moscow,152.5,1.89,1.0,Over
5,06:00,Taiwan Sbl,Yulon Luxgen Dinos,Changhua,152.5,1.86,1.0,Over
6,13:00,Eua Ncaa,Yale,Brown,134.5,1.91,1.0,Over
7,14:00,Eua Ncaa,VCU Rams,Duquesne,126.5,1.80,1.0,Over
8,16:30,Eua Ncaa,Illinois,Wisconsin,148.5,1.80,1.0,Over
